In [ ]:
%matplotlib inline


Neural Networks
===============

Neural networks can be constructed using the ``torch.nn`` package.

Now that you had a glimpse of ``autograd``, ``nn`` depends on
``autograd`` to define models and differentiate them.
An ``nn.Module`` contains layers, and a method ``forward(input)``\ that
returns the ``output``.

For example, look at this network that classifies digit images:

.. figure:: /_static/img/mnist.png
   :alt: convnet

   convnet

It is a simple feed-forward network. It takes the input, feeds it
through several layers one after the other, and then finally gives the
output.

A typical training procedure for a neural network is as follows:

- Define the neural network that has some learnable parameters (or
  weights)
- Iterate over a dataset of inputs
- Process input through the network
- Compute the loss (how far is the output from being correct)
- Propagate gradients back into the network’s parameters
- Update the weights of the network, typically using a simple update rule:
  ``weight = weight - learning_rate * gradient``

Define the network
------------------

Let’s define this network:



In [41]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
#定义Net的初始化函数，这个函数定义了该神经网络的基本结构
    def __init__(self):
        super(Net, self).__init__() 
        #复制并使用Net的父类的初始化方法，即先运行nn.Module的初始化函数
        
        # 定义conv1函数的是图像卷积函数：输入为图像（1个频道，即灰度图）,输出为 6张特征图, 卷积核为3x3正方形
        self.conv1 = nn.Conv2d(1, 6, 3)
        
        # 定义conv2函数的是图像卷积函数：输入为6张特征图,输出为16张特征图, 卷积核为3x3正方形
        self.conv2 = nn.Conv2d(6, 16, 3)
        
        # 定义fc1（fullconnect）全连接函数1为线性函数：y = Wx + b，并将16*5*5个节点连接到120个节点上
        self.fc1 = nn.Linear(16 * 6 * 6, 120)  # 6*6 from image dimension 
        
        #定义fc2（fullconnect）全连接函数2为线性函数：y = Wx + b，并将120个节点连接到84个节点上
        self.fc2 = nn.Linear(120, 84)
        
        #定义fc3（fullconnect）全连接函数3为线性函数：y = Wx + b，并将84个节点连接到10个节点上
        self.fc3 = nn.Linear(84, 10)

        
    #定义该神经网络的向前传播函数，该函数必须定义，一旦定义成功，向后传播函数也会自动生成（autograd）
    def forward(self, x):
        #输入x经过卷积conv1之后，经过激活函数ReLU，使用2x2的窗口进行最大池化Max pooling，然后更新到x
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        
        #输入x经过卷积conv2之后，经过激活函数ReLU，使用2x2的窗口进行最大池化Max pooling，然后更新到x
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        
        #view函数将张量x变形成一维的向量形式，总特征数并不改变，为接下来的全连接作准备
        x = x.view(-1, self.num_flat_features(x)) #扁平化特征
        
        #输入x经过全连接1，再经过ReLU激活函数，然后更新x
        x = F.relu(self.fc1(x))
        
        #输入x经过全连接2，再经过ReLU激活函数，然后更新x
        x = F.relu(self.fc2(x))
        
        #输入x经过全连接3，然后更新x
        x = self.fc3(x)
        return x
    
    
        """
        使用num_flat_features函数计算张量x的总特征量
        （把每个数字都看出是一个特征，即特征总量），
        比如x是4*2*2的张量，那么它的特征总量就是16。
        """    
    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        """
        这里为什么要使用[1:],是因为pytorch只接受批输入，
        也就是说一次性输入好几张图片，那么输入数据张量的维度自然上升到了4维。
        【1:】让我们把注意力放在后3维上面.(nSamples x nChannels x Height x Width)
        """
        #print("size = ",size)
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=576, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)


You just have to define the ``forward`` function, and the ``backward``
function (where gradients are computed) is automatically defined for you
using ``autograd``.
You can use any of the Tensor operations in the ``forward`` function.

The learnable parameters of a model are returned by ``net.parameters()``



In [25]:
params = list(net.parameters())  #模型的可学习参数由返回 net.parameters()
print(len(params))
print(params[0].size())  # conv1's .weight

10
torch.Size([6, 1, 3, 3])


Let try a random 32x32 input.
Note: expected input size of this net (LeNet) is 32x32. To use this net on
MNIST dataset, please resize the images from the dataset to 32x32.



In [26]:
input = torch.randn(1, 1, 32, 32) # nSamples x nChannels x Height x Width
out = net(input)
print(out)

size =  torch.Size([16, 6, 6])
tensor([[-0.1000, -0.0780,  0.0724,  0.2391,  0.1275, -0.1107, -0.0867,  0.0717,
          0.1152,  0.1348]], grad_fn=<AddmmBackward>)


In [36]:
input.size()

torch.Size([1, 1, 32, 32])

Zero the gradient buffers of all parameters and backprops with random
gradients:



In [27]:
net.zero_grad()
out.backward(torch.randn(1, 10)) #标准正态分布

<div class="alert alert-info"><h4>Note</h4><p>``torch.nn`` only supports mini-batches. The entire ``torch.nn``
    package only supports inputs that are a mini-batch of samples, and not
    a single sample.

    For example, ``nn.Conv2d`` will take in a 4D Tensor of
    ``nSamples x nChannels x Height x Width``.

    If you have a single sample, just use ``input.unsqueeze(0)`` to add
    a fake batch dimension.</p></div>

Before proceeding further, let's recap all the classes you’ve seen so far.

**Recap:**
  -  ``torch.Tensor`` - A *multi-dimensional array* with support for autograd
     operations like ``backward()``. Also *holds the gradient* w.r.t. the
     tensor.
  -  ``nn.Module`` - Neural network module. *Convenient way of
     encapsulating parameters*, with helpers for moving them to GPU,
     exporting, loading, etc.
  -  ``nn.Parameter`` - A kind of Tensor, that is *automatically
     registered as a parameter when assigned as an attribute to a*
     ``Module``.
  -  ``autograd.Function`` - Implements *forward and backward definitions
     of an autograd operation*. Every ``Tensor`` operation creates at
     least a single ``Function`` node that connects to functions that
     created a ``Tensor`` and *encodes its history*.

**At this point, we covered:**
  -  Defining a neural network
  -  Processing inputs and calling backward

**Still Left:**
  -  Computing the loss
  -  Updating the weights of the network

Loss Function
-------------
A loss function takes the (output, target) pair of inputs, and computes a
value that estimates how far away the output is from the target.

There are several different
`loss functions <https://pytorch.org/docs/nn.html#loss-functions>`_ under the
nn package .
A simple loss is: ``nn.MSELoss`` which computes the mean-squared error
between the input and the target.

For example:



损失函数采用一对（输出，目标）输入，并计算一个值，该值估计输出与目标的距离。

nn软件包下有几种不同的 损失函数。一个简单的损失是：nn.MSELoss计算输入和目标之间的均方误差。

## view中的参数-1是什么意思？
如果有任何情况您不知道要多少行，但是确定了列数，则可以使用-1来指定。（请注意，您可以将其扩展到具有更大尺寸的张量。只有一个轴值可以为-1）。这是一种告诉库的方法：“给我一个具有这么多列的张量，然后您计算出实现这一点所必需的适当行数”。

In [52]:
output = net(input)
target = torch.randn(10)  # 创建一个虚构目标
target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()   #定义损失函数

loss = criterion(output, target)
print(loss)

tensor(0.6245, grad_fn=<MseLossBackward>)


In [37]:
target.size() #返回张量尺寸

torch.Size([1, 10])

Now, if you follow ``loss`` in the backward direction, using its
``.grad_fn`` attribute, you will see a graph of computations that looks
like this:

::

    input -> conv2d -> relu -> maxpool2d -> conv2d -> relu -> maxpool2d
          -> view -> linear -> relu -> linear -> relu -> linear
          -> MSELoss
          -> loss

So, when we call ``loss.backward()``, the whole graph is differentiated
w.r.t. the loss, and all Tensors in the graph that has ``requires_grad=True``
will have their ``.grad`` Tensor accumulated with the gradient.

For illustration, let us follow a few steps backward:



In [44]:
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

Backprop
--------
To backpropagate the error all we have to do is to ``loss.backward()``.
You need to clear the existing gradients though, else gradients will be
accumulated to existing gradients.


Now we shall call ``loss.backward()``, and have a look at conv1's bias
gradients before and after the backward.



In [45]:
net.zero_grad()     # 需要清除现有的渐变，否则渐变将累积到现有的渐变中

print('conv1.bias.grad before backward')
print(net.conv1.bias.grad)

loss.backward()

print('conv1.bias.grad after backward')
print(net.conv1.bias.grad)

conv1.bias.grad before backward
None
conv1.bias.grad after backward
tensor([ 0.0064,  0.0012, -0.0127,  0.0126,  0.0011,  0.0006])


Now, we have seen how to use loss functions.

**Read Later:**

  The neural network package contains various modules and loss functions
  that form the building blocks of deep neural networks. A full list with
  documentation is `here <https://pytorch.org/docs/nn>`_.

**The only thing left to learn is:**

  - Updating the weights of the network

Update the weights
------------------
The simplest update rule used in practice is the Stochastic Gradient
Descent (SGD):

     ``weight = weight - learning_rate * gradient``

We can implement this using simple python code:

.. code:: python

    learning_rate = 0.01
    for f in net.parameters():
        f.data.sub_(f.grad.data * learning_rate)

However, as you use neural networks, you want to use various different
update rules such as SGD, Nesterov-SGD, Adam, RMSProp, etc.
To enable this, we built a small package: ``torch.optim`` that
implements all these methods. Using it is very simple:



In [46]:
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)

# in your training loop:
optimizer.zero_grad()   # zero the gradient buffers
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step()    # Does the update

.. Note::

      Observe how gradient buffers had to be manually set to zero using
      ``optimizer.zero_grad()``. This is because gradients are accumulated
      as explained in `Backprop`_ section.

